## Pipeline Jobs

## Job status Tenant

In [ ]:

node('vdr') {
    
    //stage('Cleanup') {
    //    deleteDir()
    //}
    
    //stage("checkout feature/seen-2016 branch git"){
    //    checkout([$class: 'GitSCM', branches: [[name: '*/feature/SEEN-2016']], doGenerateSubmoduleConfigurations: false, extensions: [], submoduleCfg: [], userRemoteConfigs: [[credentialsId: '5ce92219-3109-476e-9ff1-6c9b73a988c7', url: 'ssh://gitolite@www.forge.orange-labs.fr/seen/seen-devops-scripts.git']]])
    //}
    
    //stage("get workspace from tools_bachus"){
    //    def sas_ip = "10.253.52.106"
    //    def sas_ssh_user = "seen-devel"
    //    sh " scp ${sas_ssh_user}@${sas_ip}:${VERSION}  . " 
    //    sh """
    //        tar xf \$( basename ${VERSION} ) && cd \$( basename ${VERSION} .tar.gz )
    //        ls --all
    //        cd tools_jenkins/
    //        ls --all
    //      """
    //}
  
    stage("build script "){
        sh """
            ls --all
            set +x
            echo $ENV_NAME
            echo $TENANT
            echo $CUSTOMER_ID
            echo $POP
            cd tools_jenkins/status_tenant/
            python main.py --env $ENV_NAME --tenant $TENANT --customer_id $CUSTOMER_ID --jenkins_node "vdr"
            
        """
        stash name: "job_ws", includes: ""

    }
}

stage "second step on second node"

// Run on a node with the "second-node" label.
node('sas') {
        stage('Cleanup') {
            deleteDir()
        }
       
        unstash "job_ws"
    
        // Look, no output directory under the root!
        // pwd() outputs the current directory Pipeline is running in.
        sh "ls -la ${pwd()}"
    
        sh """
            cd ${pwd()}/tools_jenkins/status_tenant/
            
            python main.py --env $ENV_NAME --tenant $TENANT --customer_id $CUSTOMER_ID --jenkins_node "bachus" $POP

        """
        
        //if (currentBuild.currentResult == "SUCCESS") { 
        //    emailext attachmentsPattern: '**/view.html', body: 'Find job result in attachments', subject: 'job result', to: 'arcadius.soglo@orange.com'
        //}
        
    stash name: "job_ws_result", includes: ""

}

stage "Report delivery, on node vdr"
//on node vdr because bachus do not have access to slack, firewall reasons
node('vdr') {
    
    stage('Cleanup') {
        deleteDir()
    }
    unstash "job_ws_result"
    
    // Look, no output directory under the root!
    // pwd() outputs the current directory Pipeline is running in.
    sh "ls -la ${pwd()}"
    
    sh """
        cd ${pwd()}/tools_jenkins/status_tenant/
        python main.py --customer_id $CUSTOMER_ID --env $ENV_NAME --tenant $TENANT --slack True
    """
    def sas_ip = "10.253.52.106"
    def sas_ssh_user = "seen-devel"
    def delivery_folder = "/srv/seen-devel/jenkins/sftp/files/status_tenant"
    sh """
        scp ${pwd()}/tools_jenkins/status_tenant/html/report.html  ${sas_ssh_user}@${sas_ip}:${delivery_folder}
    """ 
}

# Job clean bdd front 

In [ ]:
set +x

scp seen-devel@10.253.52.106:$VERSION .
tar xf $(basename $VERSION) && cd $(basename $VERSION .tar.gz)

cd clean_bdd_front

chmod u+x ./setup.sh


if [ "SEEN_PIC" = "$ENV_NAME" ]; then
	export ENV_IP="10.238.110.62"
    export PORT="3306"
fi

if [ "SEEN_QA" = "$ENV_NAME" ]; then
	export ENV_IP="10.238.110.62"
    export PORT="3307"
fi

if [ "SEEN_UAT" = "$ENV_NAME" ]; then
	export ENV_IP="10.238.110.83"
    export PORT="3306"
fi

export CUSTOMER_ID="$CUSTOMER_ID"
export CUSTOMER_SITE_NAME="$CUSTOMER_SITE_NAME"

#####  CONDITIONAL RUNS => RUN_TEST SELECTED (TRUE) DEFAULT CHOICE

if [ "true" = "$RUN_TEST" ]; then

	echo "**********************************************"
	echo "**********************************************"
	echo " ************** MODE TEST ********************"
	echo "**********************************************"
	echo "**********************************************"
    
    export ACTION=$(echo $ACTION | sed s/'delete'/'select'/ )
    
	./setup.sh 

fi

## ###  CONDITIONAL RUNS => RUN_TEST NOT SELECTED (FALSE)
if [ "false" = "$RUN_TEST" ]; then

	echo "**********************************************"
	echo "**********************************************"
	echo " *************** MODE RUN ********************"
	echo "**********************************************"
	echo "**********************************************"
    
    export ACTION_DELETE=$ACTION
 
    export ACTION=$(echo $ACTION | sed s/'delete'/'select'/ )

	./setup.sh
    
	export ACTION=$ACTION_DELETE

	./setup.sh 
    
fi